In [1]:
# I use case 5 as an example.

In [2]:
import sys

import numpy as np

sys.path.append('..')

In [3]:
# case_id = 1
# case_id = 2
# case_id = 3
# case_id = 4
case_id = 5
# case_id = 6

In [4]:
labels = np.load("./group_idx_fg_{:02d}.npy".format(case_id))
np.unique(labels)

array([0, 1, 2, 3, 4, 5, 6, 7], dtype=int32)

In [5]:
traj = np.load("./traj_{:02d}.npy".format(case_id))
traj.shape

(1606, 8, 2)

In [6]:
from multiviewmoseg.transform import MultiFrameFundamentalMatrixTransform as MFT

In [7]:
# 0 and 2 are from two different moving group
# 2, 4
# 3
# 0,1,5,6,7

In [8]:
mask_a = labels == 3
# mask_b = labels == 1
mask_b = labels == 1
mask_comb = mask_a + mask_b
print(mask_a.sum(), mask_b.sum(), mask_comb.sum())
temp_trans_a = MFT(n_frames=8)
temp_trans_b = MFT(n_frames=8)
temp_trans_comb = MFT(n_frames=8)

temp_trans_a.fit(data=traj[mask_a])
temp_trans_b.fit(data=traj[mask_b])

# data inbalance issue solve by extra sampling on
if mask_a.sum() > mask_b.sum():
    idx_a = np.arange(mask_a.sum())
    idx_b = np.concatenate(
        [
            np.arange(mask_b.sum()),
            np.random.choice(
                np.arange(mask_b.sum()),
                size=mask_a.sum() - mask_b.sum(),
                replace=True,
            ),
        ]
    )
elif mask_a.sum() < mask_b.sum():
    idx_a = np.concatenate(
        [
            np.arange(mask_a.sum()),
            np.random.choice(
                np.arange(mask_a.sum()),
                size=mask_b.sum() - mask_a.sum(),
                replace=True,
            ),
        ]
    )
    idx_b = np.arange(mask_b.sum())
else:
    idx_a = np.arange(mask_a.sum())
    idx_b = np.arange(mask_b.sum())


print(idx_a.shape, idx_b.shape)


temp_trans_comb.fit(data=traj[np.concatenate([idx_a, idx_b])])
# temp_trans_comb.fit(data=traj[mask_comb])

232 173 405
(232,) (232,)


In [9]:
# self fitting error
res_data_a_model_a = temp_trans_a.residuals(data=traj[mask_a])
print(res_data_a_model_a.mean(), res_data_a_model_a.std())
res_data_b_model_b = temp_trans_b.residuals(data=traj[mask_b])
print(res_data_b_model_b.mean(), res_data_b_model_b.std())

# fitting error
res_data_b_model_a = temp_trans_a.residuals(data=traj[mask_b])
print(res_data_b_model_a.mean(), res_data_b_model_a.std())
res_data_a_model_b = temp_trans_b.residuals(data=traj[mask_a])
print(res_data_a_model_b.mean(), res_data_a_model_b.std())

# conclusion: do not use data from one single set to fit another, the degeneracy might lead to fake high error

0.5285771108632819 0.24692646262083673
0.7387615889491771 0.36130511811486066
6.6652446874684195 1.1715773671148029
26.054554276045117 10.98448674995414


In [10]:
res_data_comb_model_a = temp_trans_comb.residuals(data=traj[mask_a])
print(res_data_comb_model_a.mean(), res_data_comb_model_a.std())

res_data_comb_model_b = temp_trans_comb.residuals(data=traj[mask_b])
print(res_data_comb_model_b.mean(), res_data_comb_model_b.std())

res_data_comb_model_comb = temp_trans_comb.residuals(data=traj[mask_comb])
print(res_data_comb_model_comb.mean(), res_data_comb_model_comb.std())

29.22921435095516 5.735751566852195
16.43076100456525 2.5186710840094193
23.76222069928737 7.850896896004689


In [11]:
print(res_data_comb_model_a.mean() / res_data_a_model_a.mean())
print(res_data_comb_model_b.mean() / res_data_b_model_b.mean())

55.2979191687235
22.24095195303339


In [12]:
# 1.32 is a threshould

In [13]:
# model imbalance issue